**Mufti Alfarokhul Azam <br> 510317**

In [170]:
import mysql.connector
import pandas as pd
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [171]:
db = mysql.connector.connect(
  host="localhost",
  user="muftialroaz",
  password="password",
  database="rekmed",
  port=3306
)

Rekam Medis

In [172]:
# Retrieve all records from the 'customers' table
cursor = db.cursor()
cursor.execute("SELECT rekam_medis.rm_id, rm_diagnosis.nama_diagnosis, dokter.user_id AS dokter_user_id, dokter.nama AS dokter_nama, user.username AS dokter_username, pasien.mr AS pasien_mr, pasien.nama AS pasien_nama FROM rekam_medis LEFT JOIN dokter ON rekam_medis.user_id = dokter.user_id LEFT JOIN rm_diagnosis ON rekam_medis.rm_id = rm_diagnosis.rm_id LEFT JOIN pasien ON rekam_medis.mr = pasien.mr LEFT JOIN user ON dokter.user_id = user.id;")

# Fetch all results as a list of tuples
rekam_medis = cursor.fetchall()
rekam_medis = pd.DataFrame(rekam_medis)
rekam_medis.rename(columns={0: 'rm_id', 1: 'diagnosis_nama', 2:"dokter_user_id", 3: 'dokter_nama', 4: "dokter_username", 5: "pasien_mr", 6:"pasien_nama"}, inplace=True)

cursor.close()
db.close()

In [173]:
from pasien import pasien_data
from diagnosis import diagnosis_data
from dokter import dokter_data
from modules import preprocess_input
import db

input_data = input()
search = preprocess_input(input_data)
nama_pasien = pasien_data(search)
diagnosis = diagnosis_data(search)
nama_dokter = dokter_data(search)
rekam_medis = db.rekam_medis.copy()

In [174]:
rekam_medis

,rm_id,diagnosis_nama,dokter_user_id,dokter_nama,dokter_username,pasien_mr,pasien_nama,tanggal_periksa
0,1,influenza,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-08-10
1,2,Kosong,112,Yudhistya,huroncom@gmail.com,108000002,Dian Kusuma,2016-08-10
2,3,Kosong,112,Yudhistya,huroncom@gmail.com,108000003,Ahmad Surya,2016-08-10
3,4,Kosong,112,Yudhistya,huroncom@gmail.com,108000002,Dian Kusuma,2016-08-11
4,5,Kosong,112,Yudhistya,huroncom@gmail.com,108000002,Dian Kusuma,2016-08-11
...,...,...,...,...,...,...,...,...
284,285,Kosong,114,dr Yudhistya SpOG,indosehat,109000071,Noah Lee,2016-11-30
285,286,Kosong,114,dr Yudhistya SpOG,indosehat,109000070,Ava Santoso,2016-11-30
286,287,Kosong,114,dr Yudhistya SpOG,indosehat,109000069,Jack Wilson,2016-11-30
287,288,Kosong,114,dr Yudhistya SpOG,indosehat,109000105,Aaliyah Davis,2016-11-30


In [175]:
rekam_medis['cosine_diagnosis'] = diagnosis['cosine']
rekam_medis['cosine_dokter'] = nama_dokter['cosine']
rekam_medis['cosine_pasien'] = nama_pasien['cosine']
# rekam_medis['cosine'] = (nama_pasien['cosine'] + diagnosis['cosine'] + nama_dokter['cosine'])/3
# Menggabungkan konten dari tiga kolom ke satu kolom baru 'combined'
rekam_medis['combined_text'] = rekam_medis['pasien_nama'] + ' ' + rekam_medis['dokter_nama'] + ' ' + rekam_medis['diagnosis_nama']
rekam_medis.sort_values(by='cosine_diagnosis', ascending=False)

# diagnosis_nama = rekam_medis.loc[rekam_medis['cosine'] > 0, 'diagnosis_nama']
# pasien_nama = rekam_medis.loc[rekam_medis['cosine'] > 0, 'pasien_nama']
# dokter_nama = rekam_medis.loc[rekam_medis['cosine'] > 0, 'dokter_nama']

,rm_id,diagnosis_nama,dokter_user_id,dokter_nama,dokter_username,pasien_mr,pasien_nama,tanggal_periksa,cosine_diagnosis,cosine_dokter,cosine_pasien,combined_text
145,146,ispa,93,anita,saskyagitaamira@yahoo.com,94000010,Lina Rahayu,2016-10-27,1.0,1.0,0.0,Lina Rahayu anita ispa
40,40,ispa,88,abs,rifkizakariadr@gmail.com,89000001,Rina Cahyani,2016-08-31,1.0,0.0,0.0,Rina Cahyani abs ispa
41,41,ispa,88,abs,rifkizakariadr@gmail.com,89000001,Rina Cahyani,2016-08-31,1.0,0.0,0.0,Rina Cahyani abs ispa
152,153,ispa,93,anita,saskyagitaamira@yahoo.com,94000016,Nita Setiawan,2016-10-27,1.0,1.0,0.0,Nita Setiawan anita ispa
154,155,ispa,93,anita,saskyagitaamira@yahoo.com,94000018,Maya Prasetyo,2016-10-27,1.0,1.0,0.0,Maya Prasetyo anita ispa
...,...,...,...,...,...,...,...,...,...,...,...,...
103,103,Kosong,112,Yudhistya,huroncom@gmail.com,108000007,Dodi Prasetyo,2016-10-06,0.0,0.0,0.0,Dodi Prasetyo Yudhistya Kosong
104,104,Kosong,126,Supardi,chupa.supardi@gmail.com,117000001,Noah Hidayat,2016-10-06,0.0,0.0,0.0,Noah Hidayat Supardi Kosong
105,105,hyperemesis,39,Dr Hendry,drhendryadi@yahoo.com,40000001,Arif Hidayat,2016-10-06,0.0,0.0,0.0,Arif Hidayat Dr Hendry hyperemesis
106,106,Kosong,114,dr Yudhistya SpOG,indosehat,109000033,Aria Davis,2016-10-04,0.0,0.0,0.0,Aria Davis dr Yudhistya SpOG Kosong


In [176]:
import pandas as pd
from modules import preprocess, vektorisasi, cosine_sim, bm_25

doc_pre = preprocess(rekam_medis['combined_text'])

doc_vec, search_vec = vektorisasi(doc_pre, search)
cosi = cosine_sim(doc_vec, search_vec)

  # gabungkan dalam satu dataframe
df_vector = pd.concat([rekam_medis, cosi], axis=1) 
df_vector.sort_values(by='cosine', ascending=False)


,rm_id,diagnosis_nama,dokter_user_id,dokter_nama,dokter_username,pasien_mr,pasien_nama,tanggal_periksa,cosine_diagnosis,cosine_dokter,cosine_pasien,combined_text,cosine
152,153,ispa,93,anita,saskyagitaamira@yahoo.com,94000016,Nita Setiawan,2016-10-27,1.0,1.0,0.0,Nita Setiawan anita ispa,0.707107
154,155,ispa,93,anita,saskyagitaamira@yahoo.com,94000018,Maya Prasetyo,2016-10-27,1.0,1.0,0.0,Maya Prasetyo anita ispa,0.707107
145,146,ispa,93,anita,saskyagitaamira@yahoo.com,94000010,Lina Rahayu,2016-10-27,1.0,1.0,0.0,Lina Rahayu anita ispa,0.707107
144,145,Kosong,93,anita,saskyagitaamira@yahoo.com,94000009,Rendi Pratama,2016-10-27,0.0,1.0,0.0,Rendi Pratama anita Kosong,0.353553
192,193,Kosong,93,anita,saskyagitaamira@yahoo.com,94000058,Siti Wulandari,2016-10-28,0.0,1.0,0.0,Siti Wulandari anita Kosong,0.353553
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,98,Kosong,230,andiangga85@gmail.com,andiangga85@gmail.com,225000002,Isabella Kusuma,2016-10-05,0.0,0.0,0.0,Isabella Kusuma andiangga85@gmail.com Kosong,0.000000
99,99,Kosong,230,andiangga85@gmail.com,andiangga85@gmail.com,225000005,Rizki Cahyadi,2016-10-05,0.0,0.0,0.0,Rizki Cahyadi andiangga85@gmail.com Kosong,0.000000
100,100,"Haemophilus influenzae infection, unspecified",230,andiangga85@gmail.com,andiangga85@gmail.com,225000004,Rini Wulandari,2016-10-05,0.0,0.0,0.0,Rini Wulandari andiangga85@gmail.com Haemophil...,0.000000
101,101,Kosong,112,Yudhistya,huroncom@gmail.com,108000008,Nurul Hikmah,2016-09-24,0.0,0.0,0.0,Nurul Hikmah Yudhistya Kosong,0.000000


In [120]:
import pandas as pd

# Contoh data hasil pencarian
data = {
    'cosine_diagnosis': rekam_medis['cosine_diagnosis'].tolist(),
    'cosine_dokter': rekam_medis['cosine_dokter'].tolist(),
    'cosine_pasien': rekam_medis['cosine_pasien'].tolist()
}

df = pd.DataFrame(data)

def evaluate_top_k(df, column_name, k):
    # Urutkan berdasarkan nilai cosine similarity dari tertinggi ke terendah
    df_sorted = df.sort_values(by=column_name, ascending=False)
    
    # Ambil top-k hasil
    top_k_results = df_sorted.head(k)
    
    # Hitung metrik evaluasi
    relevant_count = top_k_results[top_k_results[column_name] > 0].shape[0]
    precision = relevant_count / k
    recall = relevant_count / df[df[column_name] > 0].shape[0] if df[df[column_name] > 0].shape[0] > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1


print("Evaluasi dengan keyword:", input_data)
# Evaluasi untuk kolom cosine_diagnosis
precision_diag, recall_diag, f1_diag = evaluate_top_k(df, 'cosine_diagnosis', 20)
print("Cosine Diagnosis:")
print("Precision:", precision_diag)
print("Recall:", recall_diag)
print("F1-Score:", f1_diag)

# Evaluasi untuk kolom cosine_dokter
precision_dokter, recall_dokter, f1_dokter = evaluate_top_k(df, 'cosine_dokter', 20)
print("\nCosine Dokter:")
print("Precision:", precision_dokter)
print("Recall:", recall_dokter)
print("F1-Score:", f1_dokter)

# Evaluasi untuk kolom cosine_pasien
precision_pasien, recall_pasien, f1_pasien = evaluate_top_k(df, 'cosine_pasien', 20)
print("\nCosine Pasien:")
print("Precision:", precision_pasien)
print("Recall:", recall_pasien)
print("F1-Score:", f1_pasien)


Evaluasi dengan keyword: anita ispa
Cosine Diagnosis:
Precision: 0.25
Recall: 1.0
F1-Score: 0.4

Cosine Dokter:
Precision: 1.0
Recall: 0.24390243902439024
F1-Score: 0.39215686274509803

Cosine Pasien:
Precision: 0.05
Recall: 1.0
F1-Score: 0.09523809523809523


In [127]:
import pandas as pd

# Contoh data hasil pencarian
data = {
    'rm_id': [145, 40, 41, 152, 154, 103, 104, 105, 106, 288],
    'cosine_diagnosis': [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    'cosine_dokter': [1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    'cosine_pasien': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
}

df = pd.DataFrame(data)

def evaluate_top_k(df, column_name, k):
    # Urutkan berdasarkan nilai cosine similarity dari tertinggi ke terendah
    df_sorted = df.sort_values(by=column_name, ascending=False)
    
    # Ambil top-k hasil
    top_k_results = df_sorted.head(k)
    
    # Hitung metrik evaluasi
    precision = top_k_results[top_k_results[column_name] > 0].shape[0] / k
    recall = top_k_results[top_k_results[column_name] > 0].shape[0] / df.shape[0] if df.shape[0] > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Evaluasi untuk kolom cosine_diagnosis
precision_diag, recall_diag, f1_diag = evaluate_top_k(df, 'cosine_diagnosis', 5)
print("Cosine Diagnosis:")
print("Precision:", precision_diag)
print("Recall:", recall_diag)
print("F1-Score:", f1_diag)

# Evaluasi untuk kolom cosine_dokter
precision_dokter, recall_dokter, f1_dokter = evaluate_top_k(df, 'cosine_dokter', 5)
print("\nCosine Dokter:")
print("Precision:", precision_dokter)
print("Recall:", recall_dokter)
print("F1-Score:", f1_dokter)

# Evaluasi untuk kolom cosine_pasien
precision_pasien, recall_pasien, f1_pasien = evaluate_top_k(df, 'cosine_pasien', 5)
print("\nCosine Pasien:")
print("Precision:", precision_pasien)
print("Recall:", recall_pasien)
print("F1-Score:", f1_pasien)


Cosine Diagnosis:
Precision: 1.0
Recall: 0.5
F1-Score: 0.6666666666666666

Cosine Dokter:
Precision: 0.6
Recall: 0.3
F1-Score: 0.4

Cosine Pasien:
Precision: 0.0
Recall: 0.0
F1-Score: 0


In [130]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Contoh DataFrame
data = {'column1': ['This is document 1.', 'Another document here.', 'Document with some keywords.'],
        'column2': ['Document with similar keywords.', 'Some text in document 2.', 'More content in document.'],
        'column3': ['Text in the third document.', 'Document with common words.', 'Additional content.']}

df = pd.DataFrame(data)

# Kata kunci (keyword) yang akan dibandingkan
keyword = "keywords"

# Menggabungkan konten dari tiga kolom ke satu kolom baru 'combined'
df['combined'] = df['column1'] + ' ' + df['column2'] + ' ' + df['column3']

# Menghitung vektor TF-IDF untuk kata kunci dan dokumen-dokumen
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([keyword] + list(df['combined']))

# Menghitung kemiripan kosinus
cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

# Menambahkan nilai kemiripan kosinus ke DataFrame
df['cosine_similarity'] = cosine_similarities

# Menampilkan DataFrame dengan nilai kemiripan kosinus
print(df[['combined', 'cosine_similarity']])


                                            combined  cosine_similarity
0  This is document 1. Document with similar keyw...           0.196881
1  Another document here. Some text in document 2...           0.000000
2  Document with some keywords. More content in d...           0.207377


In [50]:
from rank_bm25 import BM25Okapi
import pandas as pd

# Contoh data dokumen
documents = [
    "Dokumen pertama adalah contoh dokumen",
    "Ini adalah contoh dokumen kedua",
    "Contoh yang lebih panjang untuk dokumen ketiga",
    "Ini adalah dokumen contoh terakhir"
]

# Pembentukan indeks BM25
tokenized_documents = [doc.split() for doc in documents]
bm25 = BM25Okapi(tokenized_documents)

# Contoh kueri pencarian
query = "contoh dokumen panjang"

# Hitung relevansi BM25
scores = bm25.get_scores(query.split())

# Urutkan dokumen berdasarkan skor relevansi
ranked_documents = sorted(zip(scores, documents), key=lambda x: x[0], reverse=True)

ranked = pd.DataFrame(scores)
ranked = ranked.rename(columns={0:'ranking'})
ranked
# # Tampilkan hasil
# print("Hasil Pencarian:")
# for score, doc in ranked_documents:
#     print(f"Skor: {score:.4f}, Dokumen: {doc}")

,ranking
0,0.170592
1,0.170592
2,0.827542
3,0.170592
